In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Module Input

In [ ]:
pwd

In [ ]:
import datetime
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import catboost
from catboost import Pool
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
%matplotlib inline
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## Loading data

In [ ]:
test = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv", dtype={'ID': 'int32', 'shop_id': 'int32', 'item_id': 'int32'})
item_categories = pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv", dtype={'item_category_name': 'str', 'item_category_id': 'int32'})
items = pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv", dtype={'item_name': 'str', 'item_id': 'int32', 'item_category_id': 'int32'})
shops = pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv", dtype={'shop_name': 'str', 'shop_id': 'int32'})
sales = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv", dtype={'date': 'str', 'date_block_num': 'int32', 'shop_id': 'int32', 'item_id': 'int32', 'item_price': 'float32', 'item_cnt_day': 'int32'}, 
                    parse_dates=['date'])



In [ ]:
train = sales.join(items, on='item_id', rsuffix='_tmp').join(shops, on='shop_id', rsuffix='_tmp').join(item_categories, on='item_category_id', rsuffix='_tmp').drop(columns=['item_id_tmp', 'shop_id_tmp', 'item_category_id_tmp'])

## Basic information on train data

In [ ]:
print("Top5 Record\n", train.head(),'\n')

In [ ]:
print("Basic Information")
print(train.info())

In [ ]:
print("\nData Type")
print(train.dtypes)

In [ ]:
print("\nMissing Valus")
print(train.isnull().sum())

In [ ]:
print("\nShape of Data")
print(train.shape)

In [ ]:
print("\nStats of Data")
print(train.describe())

## Remove outliers

In [ ]:
train = train[train['item_price'] > 0]

## Data preprocessing
1. remove the text features
2. aggregate the data by month

In [ ]:
train.drop(columns=['item_name', 'item_category_name', 'shop_name'], inplace=True)

In [ ]:
train_month = train.sort_values('date').groupby(['date_block_num', 'shop_id', 'item_category_id', 'item_id'], as_index=False)
train_month = train_month.agg({'item_price':['sum', 'mean'], 'item_cnt_day': ['sum', 'mean', 'count']})
train_month.columns = ['date_block_num', 'shop_id', 'item_category_id', 'item_id', 'item_price', 'mean_item_price', 'item_cnt', 'mean_item_cnt', 'transactions']

## Outliers and distribution

In [ ]:
fig, axes = plt.subplots(2,1,figsize=(8,8))
sns.boxplot(x=train_month['item_cnt'], ax=axes[0])
sns.boxplot(x=train_month['mean_item_price'], ax=axes[1])
plt.show()

In [ ]:
# remove outliers
train_month = train_month.query('item_cnt >=0 and item_cnt <= 1000 and mean_item_price < 100000')

## Creating the label
the label will be the "next month item cnt"as it is a forecast problem.

In [ ]:
train_month['item_cnt_next_month'] = train_month.sort_values('date_block_num').groupby(['shop_id', 'item_id'])['item_cnt'].shift(-1)

## Feature engineering

### Unitary item prices

In [ ]:
train_month['item_price_unit'] = train_month['item_price'] // train_month['item_cnt']
train_month['item_price_unit'].fillna(0, inplace=True)

### Group based features

In [ ]:
gp_item_price = train_month.sort_values('date_block_num').groupby(["item_id"], as_index=False).agg({'item_price':[np.min, np.max]})
gp_item_price.columns = ['item_id', 'hist_min_item_price', 'hist_max_item_price']
train_month = pd.merge(train_month, gp_item_price, on='item_id', how='left')

### Price change from its lowest and highest price

In [ ]:
train_month['price_increase'] = train_month['item_price'] - train_month['hist_min_item_price']
train_month['price_decrease'] = train_month['hist_max_item_price'] - train_month['item_price']

### Rolling window features

In [ ]:
f_min = lambda x: x.rolling(window=3, min_periods=1).min()
f_max = lambda x: x.rolling(window=3, min_periods=1).max()
f_mean = lambda x: x.rolling(window=3, min_periods=1).mean()
f_std = lambda x: x.rolling(window=3, min_periods=1).std()

function_list = [f_min, f_max, f_mean, f_std]
function_name = ['min', 'max', 'mean', 'std']

for i in range(len(function_list)):
    train_month[f'item_cnt_{function_name[i]}'] = train_month.sort_values('date_block_num').groupby(['shop_id', 'item_category_id', 'item_id'])['item_cnt'].apply(function_list[i])


In [ ]:
train_month['item_cnt_std'].fillna(0, inplace=True)

### Lag based features

In [ ]:
lag_list=[1, 2, 3]
for lag in lag_list:
    ft_name = (f'item_cnt_shifted{lag}')
    train_month[ft_name] = train_month.sort_values('date_block_num').groupby(['shop_id', 'item_category_id', 'item_id'])['item_id'].shift(lag)
    train_month[ft_name].fillna(0, inplace=True)
    
    

### Item sales count trend

In [ ]:
train_month['item_trend'] = train_month['item_cnt']
for lag in lag_list:
    ft_name = (f'item_cnt_shifted{lag}')
    train_month['item_trend'] -= train_month[ft_name]

train_month['item_trend'] /= len(lag_list) + 1

## Train/Validation Split
The first 3-28 blocks are taken as the train set, validation set are the 29-32 blocks and test set is block 33

In [ ]:
train_month['item_cnt_next_month']

In [ ]:
train_set = train_month.query('date_block_num >= 3 and date_block_num < 28')
validation_set = train_month.query('date_block_num >=28 and date_block_num < 33')
test_set = train_month.query('date_block_num == 33')

In [ ]:
train_set['item_cnt_next_month'].isna().sum()

In [ ]:
train_month.head()

In [ ]:
train_month.sort_values('date_block_num').groupby(['shop_id', 'item_id'])['item_cnt'].shift(-1)

In [ ]:
train_month[train_month['item_cnt']>200]

In [ ]:
train_month.head()

In [ ]:
train_month.head()

In [ ]:
train['date'].min().date()

In [ ]:
train['date'].max().date()